<a href="https://colab.research.google.com/github/enzocfb/general/blob/main/EDA_2023_deductivos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Analisis de Deductivos para analizar causas
Impacto :

input:
      Consolidato total de Evaluados por UGS(Formato 06) para todos los años
      Deductivos reportados por UGT
      Fechas referenciales por grupos

output:
      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"

"""

'\nAnalisis de Deductivos para analizar causas\nImpacto :\n\ninput:\n      Consolidato total de Evaluados por UGS(Formato 06) para todos los años\n      Deductivos reportados por UGT\n      Fechas referenciales por grupos\n\noutput:\n      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"\n\n'

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
ES####################################################3
# Input  / Output
####################################################3

# Archivo completo de las evaluaciones realizadas por UGS, a partir de la cual se extrae la informacion del beneficiario,
# # Alcance : REGISTRADO- desde que se inicia en ESTADO REGISTRADO, LUEGO A ESTADO EVALUADO , luego  ESTADO VIABILIDAD y finalmente # a ESTADO DE DEDUCTIVIDAD
# KEY or INDICE del ARCHIVO:   CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO)
#                              CONVENIO,  es una agrupacion de Beneficiarios (FALTA, no tiene en este momento esa data)

file = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Formato06Completo.xlsx'

# 2. ARCHIVO DE DEDUCTIVOS (BENEFICIARIO QUE ESTA EN ESTADO = "DEDUCTIVO", es aquel que "SE EXCLUYE" DEL PROGRAMA. Este archivo es recibido de  UGT que contiene la informacion de todos aquellos que fueron
# ALCANCE : VIABLE-->DEDUCTIVO
# KEY or INDICE del ARCHIVO:
#                              CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO) (SI TIENE)
#                              CONVENIO,  es una agrupacion de Beneficiarios,                                               (SI TIENE)

file_deductivos= '/content/drive/MyDrive/@@@Dev/mvcs/Data/LISTADO DE DEDUCTIVOS A (UGS).xlsx'


# Informacion a nivel de nucleos de las fechas de exp tecnico y fecha de Acta de inicio de Construccion
# Alcance : REGISTRADO-->EVALUADO-->VIABLE (TODOs: ESTADO_VIABLE = "VIABLE", INCLUIDO ESTADO_DEDUCTIVO = {"DEDUCTIVO", "NO DEDUCTIVO"] se une a nivel con archivo 2, con los
# 17 nov : Solo se tienen info para VIABLE-->DEDUCTIVO, falta a nivel de todos los VIABLES.
# KEY or INDICE del ARCHIVO:   CONVENIO,  es una agrupacion de Beneficiarios, (SI TIENE)

file_input_nucleo = '/content/drive/MyDrive/@@@Dev/mvcs/Data/NUCLEO - REPORTE COMPLEMENTARIO.xlsx'

# output temporales
file_output_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs no encontrados.xlsx'
file_output_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs si encontrados.xlsx'
file_output_3 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp.xlsx'

# output
file_output_analisis = '/content/drive/MyDrive/@@@Dev/mvcs/Data/deductivos-completo-analysis.xlsx'





In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def convert_to_date(value):
    try:
        return pd.to_datetime(value)
    except ValueError:
        # Handle the case where parsing the date fails
        return value

# # Read the Excel file with the converters parameter
# df = pd.read_excel(file, converters={"Your_Column_Name": convert_to_date})

In [ ]:
df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str})
# df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str, "FEC_EVALUACION": convert_to_date})
df.head(3)

# CRACION DE ESTADOS PARA TABLA INICIAL



,NUMERO,ORIGEN,PORTAFOLIO,PORTAFOLIO_T,GRUPO,NRO ENCUESTA,CUV,CODIGO_CCPP,UBIGEO_temp,UBIGEO,...,CRITERIO_SOCIAL,CRITERIO_TECNICO,SOCIAL ESTADO DE LA FICHA,TÉCNICA,OBSERVACIONES,FECHA CADUCIDAD CSE,FECHA_CONSULTA_CSE,ACEPTA_ENTREVISTA,ESTADO_EVALUADO_T,FEC_EVALUACION
0,1,PP068,02-2024,2024-02,2,1,0808050259454185562402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2028-08-19 00:00:00,2023-09-21 02:30:56,SI,SI,14-09-2023
1,2,PP068,02-2024,2024-02,2,2,0808050259000000002402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,SIN REGISTRO,NaN,NaN,NaN,Familia Ausente,NO,14-09-2023
2,3,PP068,02-2024,2024-02,2,3,0808050259475809342402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2024-10-09 00:00:00,2023-09-21 02:37:17,SI,SI,14-09-2023


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26450 entries, 0 to 26449
Data columns (total 54 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NUMERO                            26450 non-null  int64  
 1   ORIGEN                            26450 non-null  object 
 2   PORTAFOLIO                        26450 non-null  object 
 3   PORTAFOLIO_T                      26450 non-null  object 
 4   GRUPO                             26450 non-null  int64  
 5   NRO ENCUESTA                      26450 non-null  int64  
 6   CUV                               26450 non-null  object 
 7   CODIGO_CCPP                       26450 non-null  object 
 8   UBIGEO_temp                       26450 non-null  int64  
 9   UBIGEO                            26450 non-null  object 
 10  DEPARTAMENTO                      26450 non-null  object 
 11  PROVINCIA                         26450 non-null  object 
 12  DIST

In [ ]:
# Seleccionar campos requeridos para el analisis y construir las features

columns_selected_df = ['NUMERO', 'ORIGEN', 'PORTAFOLIO_T', 'CUV', 'CODIGO_CCPP', 'UBIGEO', 'DEPARTAMENTO','PROVINCIA', 'DISTRITO', 'CENTRO_POBLADO', 'COMUNIDAD', 'PATERNO_JEFE_HOGAR', 'NOMBRES_JEFE_HOGAR', 'DNI_JEFE_HOGAR', 'SEXO_JEFE_HOGAR', 'EDAD_JEFE_HOGAR', 'FEC_NAC_JEFE_HOGAR', 'CANT', 'CTD_DISCAPACIDAD', 'ALTURA', 'LATITUD', 'LONGITUD', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA', 'ACEPTA_ENTREVISTA', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO', 'FEC_EVALUACION', 'FECHA_CONSULTA_CSE']
df = df[columns_selected_df]

In [ ]:
print(df.info())
print(df.shape[0])

# -- 26450, esta bien, coincide con el Excel


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26450 entries, 0 to 26449
Data columns (total 31 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NUMERO                            26450 non-null  int64  
 1   ORIGEN                            26450 non-null  object 
 2   PORTAFOLIO_T                      26450 non-null  object 
 3   CUV                               26450 non-null  object 
 4   CODIGO_CCPP                       26450 non-null  object 
 5   UBIGEO                            26450 non-null  object 
 6   DEPARTAMENTO                      26450 non-null  object 
 7   PROVINCIA                         26450 non-null  object 
 8   DISTRITO                          26450 non-null  object 
 9   CENTRO_POBLADO                    26450 non-null  object 
 10  COMUNIDAD                         15300 non-null  object 
 11  PATERNO_JEFE_HOGAR                25454 non-null  object 
 12  NOMB

In [ ]:


# Creamos ESTADOS : REGISTRADO, EVALUACION,  VIABILIDAD,
constante_estado_registrado = 'REGISTRADO'

constante_estado_evaluacion_evaluado = "EVALUADO"
constante_estado_evaluacion_rechaza = "RECHAZA_EVALUACION"
constante_estado_evaluacion_otros = 'OTROS_NO_EVALUADO'

constante_estado_viabilidad_viable = "VIABLE"
constante_estado_viabilidad_no_viable = "NO VIABLE"
constante_estado_sininfo = "SIN INFORMACION"


constante_estado_deductivo = "DEDUCTIVO"
constante_estado_no_deductivo = "NO_DEDUCTIVO"

df['ESTADIO_REGISTRADO'] = constante_estado_registrado
df['ESTADO_EVALUACION'] = df['ACEPTA_ENTREVISTA'].apply(lambda x: 'SI' if x == "SI" else 'NO')

df['ESTADO_EVALUACION'] = np.where(df['ACEPTA_ENTREVISTA'] == 'SI', constante_estado_evaluacion_evaluado,
                        np.where(df['ACEPTA_ENTREVISTA'] == 'Rechazo de evaluación / Intervención', constante_estado_evaluacion_rechaza, constante_estado_evaluacion_otros))


# df['ESTADO_VIABLE'].fillna(constante_estado_viabilidad_sininfo, inplace = True)

df['ESTADO_VIABLE'] = np.where(df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable, constante_estado_viabilidad_viable,
                        np.where(df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_no_viable, constante_estado_viabilidad_no_viable, constante_estado_sininfo))



<ipython-input-11-e047dd4c8d9c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ESTADIO_REGISTRADO'] = constante_estado_registrado
<ipython-input-11-e047dd4c8d9c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ESTADO_EVALUACION'] = df['ACEPTA_ENTREVISTA'].apply(lambda x: 'SI' if x == "SI" else 'NO')
<ipython-input-11-e047dd4c8d9c>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26450 entries, 0 to 26449
Data columns (total 34 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NUMERO                            26450 non-null  int64  
 1   ORIGEN                            26450 non-null  object 
 2   PORTAFOLIO_T                      26450 non-null  object 
 3   CUV                               26450 non-null  object 
 4   CODIGO_CCPP                       26450 non-null  object 
 5   UBIGEO                            26450 non-null  object 
 6   DEPARTAMENTO                      26450 non-null  object 
 7   PROVINCIA                         26450 non-null  object 
 8   DISTRITO                          26450 non-null  object 
 9   CENTRO_POBLADO                    26450 non-null  object 
 10  COMUNIDAD                         15300 non-null  object 
 11  PATERNO_JEFE_HOGAR                25454 non-null  object 
 12  NOMB

In [ ]:
df.head(3)

,NUMERO,ORIGEN,PORTAFOLIO_T,CUV,CODIGO_CCPP,UBIGEO,DEPARTAMENTO,PROVINCIA,DISTRITO,CENTRO_POBLADO,...,VIABILIDAD_SOCIAL,VIABILIDAD_TECNICA,ACEPTA_ENTREVISTA,CRITERIO_SOCIAL,CRITERIO_TECNICO,FEC_EVALUACION,FECHA_CONSULTA_CSE,ESTADIO_REGISTRADO,ESTADO_EVALUACION,ESTADO_VIABLE
0,1,PP068,2024-02,0808050259454185562402,0808050259,080805,CUSCO,ESPINAR,PALLPATA,LLAIQUEPAMPA,...,SOCIALMENTE VIABLE,VIABLE,SI,NaN,NaN,14-09-2023,2023-09-21 02:30:56,REGISTRADO,EVALUADO,VIABLE
1,2,PP068,2024-02,0808050259000000002402,0808050259,080805,CUSCO,ESPINAR,PALLPATA,LLAIQUEPAMPA,...,NaN,NaN,Familia Ausente,NaN,NaN,14-09-2023,NaN,REGISTRADO,OTROS_NO_EVALUADO,SIN INFORMACION
2,3,PP068,2024-02,0808050259475809342402,0808050259,080805,CUSCO,ESPINAR,PALLPATA,LLAIQUEPAMPA,...,SOCIALMENTE VIABLE,VIABLE,SI,NaN,NaN,14-09-2023,2023-09-21 02:37:17,REGISTRADO,EVALUADO,VIABLE


In [ ]:
# df.to_excel(file_output_3)

In [ ]:
##############################################################
# DEDUCTIVOS
##############################################################
df_deductivos = pd.read_excel(file_deductivos)
df_nucleo = pd.read_excel(file_input_nucleo)


In [ ]:
df_nucleo.head(3)

,CONVENIO,COD UGT,FEC_APROB_EXP_TECNICO,FEC_ACTA_INICIO_CONSTRUCCION,GRUPO
0,013-2017-LOR/VMVU/PNVR,013-2017,2017-03-13,2023-06-19,GRUPO 4B
1,006-2022-PUN/VMVU/PNVR,006-2022,2022-03-25,2023-07-17,GRUPO 2
2,007-2022-PUN/VMVU/PNVR,007-2022,2022-03-25,2023-06-19,GRUPO 2


In [ ]:
df_deductivos_nucleo = pd.merge(df_deductivos, df_nucleo,  on = 'CONVENIO', how = 'left')

# ESTADO : CREAMOS EL ESTADO DEDUCTIVO PARA ESTA TABLA
df_deductivos_nucleo['ESTADO_DEDUCTIVO'] = 'DEDUCTIVO'


In [ ]:
print(df_deductivos.shape[0])
print(df_deductivos_nucleo.shape[0])
# print(df_deductivos_nucleo.head())

df_deductivos_nucleo.to_excel(file_output_3)
# print(df_deductivos_nucleo.info())

1017
1017


In [ ]:
columns_selected_deductivos = ['ID', 'CUV','DNI','CODIGO_UGT','CONVENIO','NOMBRE','AP_PATERNO','DEDUCTIVO','CARTA','MOTIVO','HT','FEC_APROB_EXP_TECNICO','FEC_ACTA_INICIO_CONSTRUCCION','ESTADO_DEDUCTIVO']

df_deductivos_nucleo = df_deductivos_nucleo[columns_selected_deductivos]
# 130 registros

In [ ]:
df_join = pd.merge(df, df_deductivos_nucleo, on = 'CUV')
df_join_left = pd.merge(df, df_deductivos_nucleo, on = 'CUV', how = 'left', suffixes=('_F06', '_deductivo'))

In [ ]:
df_join_left['FEC_EVALUACION']

0        14-09-2023
1        14-09-2023
2        14-09-2023
3        14-09-2023
4        13-09-2023
            ...    
26445    19-11-2022
26446    19-11-2022
26447    19-11-2022
26448    19-11-2022
26449    19-11-2022
Name: FEC_EVALUACION, Length: 26450, dtype: object

In [ ]:
##########################################################################3
# CALCULOS POST PROCESO
##########################################################################3

# Completar ESTADO DEDUCTIVO , luego del JOIN

df_temp = df_join_left

condition_1 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & (df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )
condition_2 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & ~(df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )

df_temp.loc[condition_1, 'ESTADO_DEDUCTIVO'] = constante_estado_no_deductivo
df_temp.loc[condition_2, 'ESTADO_DEDUCTIVO'] = constante_estado_sininfo

# FECHAS:
# FEC_EVALUACION, FEC_APROB_EXP_TECNICO,FEC_ACTA_INICIO_CONSTRUCCION
#


In [ ]:
df_temp['FEC_EVALUACION']

0        14-09-2023
1        14-09-2023
2        14-09-2023
3        14-09-2023
4        13-09-2023
            ...    
26445    19-11-2022
26446    19-11-2022
26447    19-11-2022
26448    19-11-2022
26449    19-11-2022
Name: FEC_EVALUACION, Length: 26450, dtype: object

In [ ]:
# Convert date strings to datetime objects
df_temp['FEC_EVALUACION'] = pd.to_datetime(df_temp['FEC_EVALUACION'])
df_temp['FEC_APROB_EXP_TECNICO'] = pd.to_datetime(df_temp['FEC_APROB_EXP_TECNICO'])
df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] = pd.to_datetime(df_temp['FEC_ACTA_INICIO_CONSTRUCCION'])

# Calculate the difference between the two dates
df_temp['days_diff_exp_eval'] = (df_temp['FEC_APROB_EXP_TECNICO'] - df_temp['FEC_EVALUACION']).dt.days
df_temp['days_diff_acta_eval'] = (df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] - df_temp['FEC_EVALUACION']).dt.days
df_temp['days_diff_acta_exp'] = (df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] - df_temp['FEC_APROB_EXP_TECNICO']).dt.days



<ipython-input-24-8eb3aabce75a>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_temp['FEC_EVALUACION'] = pd.to_datetime(df_temp['FEC_EVALUACION'])


In [ ]:
df_temp['days_diff_acta_exp']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
26445     0.0
26446    85.0
26447     0.0
26448     0.0
26449     0.0
Name: days_diff_acta_exp, Length: 26450, dtype: float64

In [ ]:
############################
# fillna para las fechas : se completa con 0, las que son nulas.
# 0, es un valor que en la realidad NO es valido, porque desde la EVALUACION hasta la REALIZACION del expediente, o
# DESDE la APROBACION DEL EXPEDIENTE TECNICO hasta la CONSTRUCCION, el tiempo es mayor que 0
############################

# List of columns to fill with zeros
columns_to_fill = ['days_diff_exp_eval', 'days_diff_acta_eval', 'days_diff_acta_exp']

# Fill NaN values with 0 only in the specified columns
df_temp[columns_to_fill] = df_temp[columns_to_fill].fillna(0)



In [ ]:
df_join_left = df_temp
df_join_left.to_excel(file_output_analisis)

In [ ]:
print(df.shape[0])
print(df_join.shape[0])
print(df_join_left.shape[0])
# df_join.head()
df_join_left.to_excel(file_output_analisis)

26450
130
26450


In [ ]:
df1 = df_deductivos
df2 = df

rows_not_in_df = df1[~df1['CUV'].isin(df2['CUV'])]
rows_in_df = df1[df1['CUV'].isin(df2['CUV'])]


In [ ]:
rows_not_in_df06.to_excel(file_output_1)
rows_in_df06.to_excel(file_output_2)

NameError: ignored

In [ ]:
rows_not_in_df.shape[0]

In [ ]:
import pandas as pd

# Sample DataFrames (Replace these with your actual DataFrames)
data1 = {'key': [1, 2, 3, 4, 5],
         'value': ['A', 'B', 'C', 'D', 'E']}
data2 = {'key': [2, 4, 5],
         'value': ['B', 'D', 'E']}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)
print(df1)
print(df2)

# Select rows in df1 that are not in df2 based on 'key'
rows_not_in_df2 = df1[~df1['key'].isin(df2['key'])]

print(rows_not_in_df2)
